In [23]:
# import dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import os
import pandas as pd
import time

In [24]:
# pointing to the directory where chromedriver exists
executable_path = {"executable_path":"D:\Program Files\chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless = False)

In [25]:
def get_soup_from_url(url):
    # visiting the page
    browser.visit(url)
    # using bs to write it into html
    html = browser.html
    soup = bs(html,"html.parser")
    return soup

In [12]:
##### ---------------------- For Title & Paragraph ----------------------------------------------------------
soup = get_soup_from_url("https://mars.nasa.gov/news/")
news_title = soup.find("div",class_="content_title").text
news_p = soup.find("div", class_="article_teaser_body").text
print(f"Title is: {news_title}")
print(f"Paragraph is : {news_p}")

Title is: After a Reset, Curiosity Is Operating Normally
Paragraph is : NASA's Mars rover Curiosity is in good health but takes a short break while engineers diagnose why it reset its computer. 


In [27]:
# ##### ------------------ For featured Image URL -------------------------------------------------------------
soup = get_soup_from_url("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")
href_link = soup.find("a", class_="fancybox")['data-link']
domain_link = "https://www.jpl.nasa.gov"
browser.click_link_by_id('full_image')
time.sleep(1)
try:
    new_link = domain_link + href_link
    soup_new = get_soup_from_url(new_link)
    image_src = soup_new.find("img", class_="main_image")['src']
    featured_image_url = domain_link + image_src
except:
    featured_image_url = "In featured image URL - Sorry the URL you requested not founded."
print(featured_image_url)

in try
new link is https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA11777
after soup_new

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA11777_hires.jpg


In [7]:
##### ------------------ For Mars weather -------------------------------------------------------------
# get mars weather's latest tweet from the website
soup = get_soup_from_url("https://twitter.com/marswxreport?lang=en")
mars_weather = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
text_list = mars_weather.split('https')
print(text_list[0])

Curiosity is again operating normally following a boot problem first experienced last Friday. Look for more Gale Crater weather conditions soon.



In [22]:
##### ------------------ For Mars facts -------------------------------------------------------------
# visit space facts and scrap the mars facts table
url_facts = "https://space-facts.com/mars/"
table = pd.read_html(url_facts)
df_mars_facts = table[0]
df_mars_facts.columns = ["Parameter", "Values"]
df_mars_facts.set_index(["Parameter"])
mars_facts_html = df_mars_facts.to_html(header=False, index=False)
print(mars_facts_html)

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [27]:
####### ------------------------------ Mars Hemispheres ----------------------------------------------------

hemi_lists = []

# Get 1,3,5,7 <a> tags to extract its title & url link for full image 
for i in range(1,9,2):
    hemi_dict = {}

    hemispheres_soup = get_soup_from_url('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')
    hemi_name_links = hemispheres_soup.find_all('a', class_='product-item')
    hemi_name = hemi_name_links[i].text.strip('Enhanced')

    detail_links = browser.find_by_css('a.product-item')
    detail_links[i].click()
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    time.sleep(1)
    browser.windows.current = browser.windows[-1] # Keep track of previous window
    hemi_img_html = browser.html # Set as my current browser html as current
    browser.windows.current = browser.windows[0] 
    browser.windows[-1].close() # close the child window

    hemi_img_soup = bs(hemi_img_html, 'html.parser')
    hemi_img_path = hemi_img_soup.find('img')['src']

    hemi_dict['title'] = hemi_name.strip()       
    hemi_dict['img_url'] = hemi_img_path

    hemi_lists.append(hemi_dict)

hemi_lists

   

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [28]:
 browser.quit()